In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import mybiotools as mbt
from scipy.stats import gaussian_kde
import os, sys
import gzip

# 2018-06-11 Dictionary and libraries check
I finished the scripts to generate the new dictionary. Now let's have a look at some statistics of that dictionary. The next step will be to go again to the re-processed libraries, and redo the analysis of how sane they are.

## Dictionary statistics
Let's first load the dictionary, which now contains much richer information on how that particular barcode is found.

In [ ]:
# STEP 3 : create a parser for the output file
def parse_prom_bcd_dict(prom_bcd_fname) :
    d = {}
    with open(prom_bcd_fname,'r') as f :
        for lineno,line in enumerate(f) :
            bcd,all_candidates = line.strip().split('\t')
            d_bcd = {}
            for candidates in all_candidates.split(';') :
                for candidate in candidates.split(',') :
                    prom_name,p = candidate.split(':')
                    d_bcd[prom_name] = p
            d[bcd] = d_bcd
    return d

In [ ]:
def prom_id(prom_name) :
    return prom_name[:1],prom_name[1:]

In [ ]:
pbd_fname = '%s/work/CRG/projects/hpip/data/pbd/pbd.txt'%(os.getenv('HOME'))
pbd = parse_prom_bcd_dict(pbd_fname)

In [ ]:
# let's check how much space that dictionary occupies in my memory
Mb = 2**20
sys.getsizeof(pbd)/Mb

Three gigabytes. Which is comparable to the size of the file that contains the dictionary. However, the memory footprint of the kernel shows roughly ten times more memory occupied now.

In [ ]:
print "Total: %d barcodes"%(len(pbd))
max_collisions = 96
coll_hist = np.zeros(max_collisions,dtype=np.int32)
for prom_list in pbd.itervalues() :
    nproms = len(prom_list)
    coll_hist[nproms] += 1

In [ ]:
plt_max = 10
plt.bar(np.arange(max_collisions)[:plt_max],coll_hist[:plt_max]/coll_hist.sum())
plt.xticks(np.arange(1,plt_max))
plt.xlim(0.5,plt_max+0.5)
plt.xlabel('Number of collisions')
plt.ylabel('Percentage')

Let's also have a look at another thing: for all the barcodes that collide, what are the most frequent collisions?

In [ ]:
# prepare a promoter-to-index mapping
libs = range(1,13)
prom_class_idx = {'A':0,'B':1,'C':2,'D':3,
              'E':4,'F':5,'G':6,'H':7}
prom_lib_idx = {lib : lib-1 for lib in libs}
prom_idx = {}
idx_prom = {}
for prom_class,prom_class_id in prom_class_idx.iteritems() :
    for prom_lib,prom_lib_id in prom_lib_idx.iteritems() :
        prom_name = '%s%d'%(prom_class,prom_lib)
        prom_idx[prom_name] = prom_class_id*12 + prom_lib_id
        idx_prom[prom_class_id*12 + prom_lib_id] = prom_name

In [ ]:
collision_matrix = np.zeros((96,96),dtype=np.int32)
for prom_list in pbd.itervalues() :
    nproms = len(prom_list)
    if nproms > 1 :
        proms = prom_list.keys()
        for i, prom1 in enumerate(proms) :
            for j, prom2 in enumerate(proms[i+1:]) :
                collision_matrix[
                    prom_idx[prom1],
                    prom_idx[prom2]
                ] += 1
                collision_matrix[
                    prom_idx[prom2],
                    prom_idx[prom1]
                ] += 1

In [ ]:
print prom_idx.values()[::12]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.matshow(np.log(collision_matrix+1))
vals = np.arange(0,96,12)
ticks = [idx_prom[v] for v in vals]
plt.xticks(vals,ticks)
plt.yticks(vals,ticks)
plt.show()

One has to admit that this is an awesome collision pattern. Of course this is not random and there is something strange going on. Now I group the promoters by class and look at the collisions by class.

In [ ]:
collisions_by_class = np.zeros((8,8),dtype=np.int32)
for i in xrange(8) :
    for j in xrange(i, 8) :
        I = i*12
        J = j*12
        s = collision_matrix[I:I+12,J:J+12].sum()
        collisions_by_class[i,j] = s
        collisions_by_class[j,i] = s
    collisions_by_class[i,i] /= 2.0

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
cax = ax.matshow(np.log(collisions_by_class))
plt.colorbar(cax)
plt.xticks(range(8),prom_class_idx.keys())
plt.yticks(range(8),prom_class_idx.keys())
plt.show()

In [ ]:
collisions_by_lib = np.zeros((12,12),dtype=np.int32)
for i in xrange(12) :
    for j in xrange(i, 12) :
        s = 0
        for k in xrange(8) :
            for m in xrange(k,8) :
                # print '%s %s'%(idx_prom[i+k*12],idx_prom[j+m*12])
                s += collision_matrix[i+k*12,j+m*12]
        collisions_by_lib[i,j] = s
        collisions_by_lib[j,i] = s

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
cax = ax.matshow(np.log(collisions_by_lib+1))
plt.colorbar(cax)
plt.xticks(range(12),prom_lib_idx.keys())
plt.yticks(range(12),prom_lib_idx.keys())
plt.show()

I want to zoom into the big pattern of collisions.

In [ ]:
class_i = 'C'
class_j = 'F'
I = prom_class_idx[class_i]*12
J = prom_class_idx[class_j]*12
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.matshow(np.log(collision_matrix[I:I+12,J:J+12]+1))
plt.yticks(range(12),['%s%d'%(class_i,i+1) for i in range(12)])
plt.xticks(range(12),['%s%d'%(class_j,i+1) for i in range(12)])
plt.show()

## Library sanity check (again)

In [ ]:
# generate library file names
def bcd_lib_assignment(rep,pbd) :
    mc_datadir = '/mnt/ant-login/mcorrales/HPIP/iPCR/HPIP_iPCR_%s/Data/Intensities/BaseCalls'%(rep)
    libs = range(1,13)
    prom_bcd_counts = np.zeros((12,12,8),dtype=np.int32)
    prom_class_idx = {'A':0,'B':1,'C':2,'D':3,
                  'E':4,'F':5,'G':6,'H':7}
    prom_lib_idx = {lib : lib-1 for lib in libs}
    for lib in libs :
        mbt.log_message(rep,'Parsing library %d'%lib)
        iPCR_fname = '%s/iPCR%d_S%d_R1_001.fastq.gz'%(mc_datadir,lib,lib-1)
        # skip library 1, basically
        if not os.path.exists(iPCR_fname) : continue
        # open file
        with gzip.open(iPCR_fname) as f :
            lineno = 0
            for line in f :
                lineno+=1
                # get only the sequence of the read
                if lineno%4 != 2 :
                    continue
                # get barcode
                bcd = line[:20]
                if pbd.has_key(bcd) :
                    # the barcode exists in the dictionary: fetch the list
                    # of candidate promoters
                    prom_list = pbd[bcd]
                    for prom in prom_list.iterkeys() :
                        prom_class,prom_lib = prom_id(prom)
                        prom_bcd_counts[
                            lib-1,
                            prom_lib_idx[int(prom_lib)],
                            prom_class_idx[prom_class]
                        ] += 1
    return prom_bcd_counts

In [ ]:
reps = ['rep1','rep2']
bcd_lib = {}
for rep in reps :
    bcd_lib[rep] = bcd_lib_assignment(rep,pbd)

In [ ]:
reps = ['rep1','rep2']
counts_mat = {}
for rep in reps :
    counts_mat[rep] = mbt.row_normalize_matrix(bcd_lib[rep].sum(axis=2).astype(float))

In [ ]:
for rep in reps :
    fig,ax = plt.subplots(1,1,figsize=(8,8))
    cax = ax.matshow(counts_mat[rep],cmap=plt.cm.Greens)
    cbar = plt.colorbar(cax)
    plt.xticks(range(12),[str(i+1) for i in range(12)])
    plt.yticks(range(12),[str(i+1) for i in range(12)])
    plt.xlabel('Barcode library assignment',fontsize=32)
    ax.xaxis.set_label_position('top')
    plt.ylabel('Library origin',fontsize=32)
    cbar.set_label('Frequency')
    plt.show()

Now, these are the results if we look at the non-starcoded iPCR files. What happens if we have a look at the starcoded ones?

In [ ]:
def canonical_bcd_counts(rep,pbd) :
    datadir = '%s/work/CRG/projects/hpip/data'%(os.getenv('HOME'))
    libs = range(1,13)
    cbc = np.zeros((12,12,8),dtype=np.int32)
    prom_class_idx = {'A':0,'B':1,'C':2,'D':3,
                  'E':4,'F':5,'G':6,'H':7}
    prom_lib_idx = {lib : lib-1 for lib in libs}
    for lib in libs :
        starcode_iPCR_fname = '%s/%s/lib%d/iPCR-starcode.txt'%(datadir,rep,lib)
        d = {}
        if not os.path.exists(starcode_iPCR_fname) :
            mbt.warn_message('canonical',"Skipping %s"%(starcode_iPCR_fname))
            continue
        with open(starcode_iPCR_fname,'r') as f :
            mbt.log_message('canonical','Parsing lib %d'%lib)
            for lineno,line in enumerate(f) :
                canonical,_,bcd_list_raw = line.strip('\n').split()
                bcd_list = bcd_list_raw.split(',')
                # if lineno > 10 : break
                if pbd.has_key(canonical) :
                    prom_list = pbd[canonical]
                    for prom in prom_list.iterkeys() :
                        prom_class,prom_lib = prom_id(prom)
                        cbc[
                            lib-1,
                            prom_lib_idx[int(prom_lib)],
                            prom_class_idx[prom_class]
                        ] += 1
    return cbc

In [ ]:
ccm = {}
cbc = {}
for rep in reps :
    cbc[rep] = canonical_bcd_counts(rep,pbd)
    ccm[rep] = mbt.row_normalize_matrix(cbc[rep].sum(axis=2).astype(float))

In [ ]:
for rep in reps :
    fig,ax = plt.subplots(1,1,figsize=(8,8))
    cax = ax.matshow(ccm[rep],cmap=plt.cm.Greens)
    cbar = plt.colorbar(cax)
    plt.xticks(range(12),[str(i+1) for i in range(12)])
    plt.yticks(range(12),[str(i+1) for i in range(12)])
    plt.xlabel('Barcode library assignment',fontsize=32)
    ax.xaxis.set_label_position('top')
    plt.ylabel('Library origin',fontsize=32)
    cbar.set_label('Frequency')
    plt.show()